# 02: Go Project Layout

These are notes from https://appliedgo.com/blog/go-project-layout

## Objectives 

+ Understand Go project layout best practices
+ Learn Go code project naming conventions
+ Understand the layouts for:
  + CLI tools
  + Libraries
  + Libraries with CLI tools

## Introduction

There is no such thing as an ultimate project layout. Depending on your use case you should choose a particular layout.

There is not a particular Go project layout and when an article or repo promotes a particular Go project layout as "standard" it's a framework smell.

Frameworks are unpopular in Go because they impose a rigid structure on project that might or might not match the project's specific needs.

As a result, when thinking about how to lay out a Go project apply best practices.

## Go project layout best practices

Best practices are a strong guidance without being dogmatic.


### Start small

When starting a new project and don't know how large it will become over time, use the simplest layout possible, and add structure only if required.

### Name packages by their functionality

Avoid grab-bag package names like `util`, `models`, `controllers`, `helpers`...

Instead, group your code by functionality or responsibility.

### Follow the conventions

#### Known directory names

Try to stick to the following special directory names and responsibilities:

+ `internal/`: packages inside the `internal/` directory are not accesible outside the Go module it belongs to. If you expose a package to the public, it becomes a public API, so you have to ensure it follows semver. Use the `internal/` directory to avoid this situation. Even the Go toolchain follows this convention.

+ `testdata/`: put ancillary test data inside this directory. Go toolchain will ignore anything inside it.

    Go toolchain will also ignore any directory starting with `.` or `_`.

+ `vendor/`: You can download all 3rd party dependencies into a `vendor/` directory. `go mod vendor` expects this directory name.

#### Do not use "src"

A Go project is plain source code. Therefore, there is no need to use a `src/` directory.

## Layouts by example

This is a list of examples for different use cases.

### CLI tools

CLI tools shouldn't require much structure. The simplest ones require only a single directory which will be the root of the repository.

For example, if you want to write a a tool that greets you, name the root directory `greeter/` because Go picks the main package's directory name as the name of the binary by default.

All the `*.go` files should reside in the `greeter/` directory with no additional subdirectories required.

```
greeter/
├── .gitignore
├── go.mod
├── main_test.go
├── main.go
├── Makefile
└── README.md
```

| EXAMPLE: |
| :------- |
| See [greeter](https://github.com/sergiofgonzalez/greeter) for an example. |

A variant of this approach will include an `internal/` directory for library packages that the tool uses but that you don't want to expose to the consumers (internal use libraries).

```
greeter/
├── internal/
│   └── greetings/
│       ├── greetings_test.go
│       └── greetings.go
├── .gitignore
├── go.mod
├── main_test.go
├── main.go
├── Makefile
└── README.md
```

| EXAMPLE: |
| :------- |
| See [greeter (feature/internal_lib branch)](https://github.com/sergiofgonzalez/greeter/tree/feature/internal_lib) for an example. |

Once published, users will be able to do:

```bash
go install github.com/sergiofgonzalez/greeter@latest
```

### Pure library projects

A library project with no auxiliary CLI tools can start with a single directory.

For example, a library that provides several types of greetings would look like:

```
greetings/
├── .gitignore
├── go.mod
├── greetings_test.go
├── greetings.go
├── Makefile
└── README.md
```

| EXAMPLE: |
| :------- |
| See [greetings](https://github.com/sergiofgonzalez/greetings) for an example. |

Once published, consumer programs will be able to do:

```bash
go get github.com/sergiofgonzalez/greetings
```

And use it in their programs as can be seen in [greeter (using lib branch)](https://github.com/sergiofgonzalez/greeter/tree/features/using_lib)

The library might include additional public packages as it gets more capabilities. Those should be included as separate folders at the root level.

For example, you might have a sample library that return different themed quotes. You might have some functionality that returns random quotes without caring about the topic, and specialized packages that return poetry quotes, tv quotes, etc.

```
greetings/
├── computers/
│   └── computers.go
├── poetry/
│   └── poetry.go
├── tv/
│   └── tv.go
├── .gitignore
├── go.mod
├── Makefile
├── quotes.go
└── README.md
```

| EXAMPLE: |
| :------- |
| See [quotes](https://github.com/sergiofgonzalez/quotes) for an example. |


Then consumer applications will be able to consume both the `quotes.go` capabilities, and also the specialized capabilities.


```go
import (
	"github.com/sergiofgonzalez/quotes"
	"github.com/sergiofgonzalez/quotes/computers"
	"github.com/sergiofgonzalez/quotes/poetry"
	"github.com/sergiofgonzalez/quotes/tv"  
)
```

| EXAMPLE: |
| :------- |
| See [quotes](https://github.com/sergiofgonzalez/quotes) for an example. |


### CLI tools with public packages/Libraries with CLI tools

A variant of the previous examples is a CLI tool that exposes some public packages for programmatic consumption, or alternatively, a library that also packages a CLI tool for the terminal.

This will look like the following:

```
greeter/
├── cmd/
│   └── greeter/
|        └── main.go
├── .gitignore
├── go.mod
├── greeter.go
├── Makefile
└── README.md
```

| EXAMPLE: |
| :------- |
| See [greeter (lib and CLI branch)](https://github.com/sergiofgonzalez/greeter/tree/features/lib_and_cli) for an example. |

When using this approach, consumers of the library can do:

```go
import "github.com/sergiofgonzalez/greeter"
```

And the users that would like to install the CLI tool can do:

```bash
go install github.com/sergiofgonzalez/greeter/cmd/greeter
```

### Everything together

All those previous layouts are not mutually exclusive. You can throw all together and have a library with public and internal libraries, along with CLI tools.


```
compress/
├── compress.go
├── go.mod
├── go.sum
├── encode/
│   └── encode.go
├── decode/
│   └── decode.go
├── internal/
│   └── deflate/
|        └── deflate.go
├── cmd/
│   └── compress/
|        └── main.go
├── .gitignore
├── Makefile
└── README.md
```

Note that the `internal/` directory is place at the root to make the import path shorter.